In [6]:
!pip install kaggle

In [7]:
!pip install tensorflow

In [8]:
!pip install keras

In [9]:
import os
import json

from zipfile import  ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection-Kaggle API**

In [10]:
kaggle_dictionary = json.load(open('kaggle.json'))

In [11]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [12]:
# setup Kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [13]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:01<00:00, 31.6MB/s]
100% 25.7M/25.7M [00:01<00:00, 22.1MB/s]


In [14]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [15]:
# unzip the dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [16]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


**Loading the Dataset**

In [17]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [18]:
data.shape

(50000, 2)

In [19]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [20]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [21]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [22]:
data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)

In [23]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [24]:
data['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [25]:
#split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [26]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [27]:
# Tokenize test data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [28]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [29]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [30]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [31]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**LSTM - Long Short-Term Memory**

In [32]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
#compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

**Training the Model**

In [35]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 224s 440ms/step - loss: 0.4005 - accuracy: 0.8191 - val_loss: 0.3159 - val_accuracy: 0.8696
Epoch 2/5
500/500 [==============================] - 203s 407ms/step - loss: 0.2786 - accuracy: 0.8894 - val_loss: 0.2995 - val_accuracy: 0.8739
Epoch 3/5
500/500 [==============================] - 199s 398ms/step - loss: 0.2389 - accuracy: 0.9054 - val_loss: 0.3283 - val_accuracy: 0.8687
Epoch 4/5
500/500 [==============================] - 197s 394ms/step - loss: 0.1986 - accuracy: 0.9241 - val_loss: 0.3180 - val_accuracy: 0.8744
Epoch 5/5
500/500 [==============================] - 200s 400ms/step - loss: 0.1714 - accuracy: 0.9335 - val_loss: 0.3491 - val_accuracy: 0.8730


**Model Evaluation**

In [36]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

313/313 [==============================] - 20s 61ms/step - loss: 0.3255 - accuracy: 0.8811
Test Loss: 0.3254806399345398
Test Accuracy: 0.8810999989509583


**Building a Predictive System**

In [37]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
  return sentiment

In [38]:
# example usage
new_review = 'This movie was fantastic. I loves it.'
sentiment = predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')

1/1 [==============================] - 0s 283ms/step
The sentiment of the review is: positive


In [39]:
new_review = 'This Movie was not that good'
sentiment = predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')

1/1 [==============================] - 0s 65ms/step
The sentiment of the review is: negative


In [40]:
new_review = 'This movie was ok but not that good.'
sentiment = predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')

1/1 [==============================] - 0s 63ms/step
The sentiment of the review is: negative
